In [1]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib
from load_and_preprocess import load_model_components, preprocess_data

In [2]:
# Load model components
model_components = load_model_components('models/aussie_rain.joblib')
#joblib.dump(model, 'models/aussie_rain.joblib', compress=('zlib', 3))

In [3]:
# Load dataset
raw_df = pd.read_csv('data/weatherAUS.csv')

raw_df.dropna(subset=['RainToday', 'RainTomorrow'], inplace=True)

In [4]:
# Create train and test sets
X = raw_df[model_components['input_cols']]
y = raw_df[model_components['target_col']]
train_inputs, test_inputs, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [5]:
# Check train dataset for null values
train_inputs.isna().sum().apply(lambda x: format(x/train_inputs.shape[0],'.2%'))

Location          0.00%
MinTemp           0.32%
MaxTemp           0.20%
Rainfall          0.00%
Evaporation      42.45%
Sunshine         47.55%
WindGustDir       6.51%
WindGustSpeed     6.47%
WindDir9am        6.85%
WindDir3pm        2.60%
WindSpeed9am      0.74%
WindSpeed3pm      1.79%
Humidity9am       1.07%
Humidity3pm       2.50%
Pressure9am       9.80%
Pressure3pm       9.83%
Cloud9am         37.38%
Cloud3pm         39.84%
Temp9am           0.45%
Temp3pm           1.87%
RainToday         0.00%
dtype: object

In [6]:
# Preprocess raw data with trained imputer, scaler, encoder
X_train, train_inputs = preprocess_data(train_inputs, model_components)
X_test, test_inputs = preprocess_data(test_inputs, model_components)

C:\study\07_ML_course\streamlit\load_and_preprocess.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[components['encoded_cols']] = components['encoder'].transform(data[components['categorical_cols']])
C:\study\07_ML_course\streamlit\load_and_preprocess.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[components['encoded_cols']] = components['encoder'].transform(data[components['categorical_cols']])
C:\study\07_ML_course\streamlit\load_and_preprocess.py:19: PerformanceWarning: DataFrame is highly fragmented.  Th

In [7]:
# Define and train Random Forest model
model = RandomForestClassifier(n_estimators=40, max_leaf_nodes=30, n_jobs=-1, random_state=42)
model.fit(X_train, y_train)

# Make predictions on train set
pred_train = model.predict(X_train)
print(classification_report(y_train, pred_train, digits=4))

              precision    recall  f1-score   support

          No     0.8333    0.9811    0.9012     87668
         Yes     0.8242    0.3109    0.4515     24961

    accuracy                         0.8326    112629
   macro avg     0.8288    0.6460    0.6763    112629
weighted avg     0.8313    0.8326    0.8015    112629



In [8]:
# Make predictions on test set
pred_test = model.predict(X_test)
print(classification_report(y_test, pred_test, digits=4))

              precision    recall  f1-score   support

          No     0.8344    0.9798    0.9013     21918
         Yes     0.8171    0.3171    0.4569      6240

    accuracy                         0.8329     28158
   macro avg     0.8258    0.6485    0.6791     28158
weighted avg     0.8306    0.8329    0.8028     28158



In [9]:
# Save model with new trained estimator
rf_model = {
    'model': model_components['model'],
    'imputer': model_components['imputer'],
    'scaler': model_components['scaler'],
    'encoder': model_components['encoder'],
    'input_cols': model_components['input_cols'],
    'target_col': model_components['target_col'],
    'numeric_cols': model_components['numeric_cols'],
    'categorical_cols': model_components['categorical_cols'],
    'encoded_cols': model_components['encoded_cols']
}
joblib.dump(rf_model, "models/rf_model.joblib")

['models/rf_model.joblib']

In [10]:
raw_df.describe()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm
count,140319.000000,140480.00000,140787.000000,81093.000000,73982.000000,131682.000000,139732.000000,138256.000000,139270.000000,137286.000000,127044.000000,127018.000000,88162.000000,84693.000000,140131.000000,138163.000000
mean,12.184824,23.23512,2.349974,5.472516,7.630540,39.970520,13.990496,18.631141,68.826833,51.449288,1017.654577,1015.257963,4.431161,4.499250,16.987066,21.693183
std,6.403879,7.11450,8.465173,4.189132,3.781729,13.578201,8.886210,8.798096,19.063650,20.807310,7.104867,7.035411,2.886594,2.719752,6.496012,6.937784
min,-8.500000,-4.80000,0.000000,0.000000,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,980.500000,977.100000,0.000000,0.000000,-7.200000,-5.400000
25%,7.600000,17.90000,0.000000,2.600000,4.900000,31.000000,7.000000,13.000000,57.000000,37.000000,1013.000000,1010.400000,1.000000,2.000000,12.300000,16.600000
50%,12.000000,22.60000,0.000000,4.800000,8.500000,39.000000,13.000000,19.000000,70.000000,52.000000,1017.600000,1015.200000,5.000000,5.000000,16.700000,21.100000
75%,16.800000,28.30000,0.800000,7.400000,10.700000,48.000000,19.000000,24.000000,83.000000,66.000000,1022.400000,1020.000000,7.000000,7.000000,21.600000,26.400000
max,33.900000,48.10000,371.000000,145.000000,14.500000,135.000000,130.000000,87.000000,100.000000,100.000000,1041.000000,1039.600000,9.000000,9.000000,40.200000,46.700000
